In [1]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt 
from pandas import read_csv
import pandas as pd 
import numpy as np

In [2]:
# define dataset
train_info = read_csv('train.csv')
meal_info = read_csv('meal_info.csv')
fulfilment_center_info = read_csv('fulfilment_center_info.csv')

# one-hot encoding
meal_info_1 =  pd.get_dummies(meal_info)
fulfilment_center_info_1 =  pd.get_dummies(fulfilment_center_info)
print(meal_info_1.head())
print(fulfilment_center_info_1.head())

# Labels are the values we want to predict
labels = np.array(train_info['num_orders'])

# taking one center's data (id=55)
c = 55 #center id
feature_list = list(train_info.columns)
train_info.nunique(axis = 0) 
center_list = train_info['center_id'].unique()
train_center_c = train_info.loc[train_info['center_id']==c]

# # Convert to numpy array
# train_np = np.array(train)
#print(train_center_c['meal_id'].unique() ) #gives different meal_id. select id = 1885

m = 1885
center_c_meal_m = train_center_c.loc[train_center_c['meal_id'] == m]




   meal_id  category_Beverages  category_Biryani  category_Desert  \
0     1885                   1                 0                0   
1     1993                   1                 0                0   
2     2539                   1                 0                0   
3     1248                   1                 0                0   
4     2631                   1                 0                0   

   category_Extras  category_Fish  category_Other Snacks  category_Pasta  \
0                0              0                      0               0   
1                0              0                      0               0   
2                0              0                      0               0   
3                0              0                      0               0   
4                0              0                      0               0   

   category_Pizza  category_Rice Bowl  category_Salad  category_Sandwich  \
0               0                   0               

In [3]:
#making a new dataframe with current features and added n lag values of num_orders and splitting into train-test
def new_dataset(center_c_meal_m,n_lag,n_split):
    new_train_data = center_c_meal_m
    
    for i in range(n_lag):
        new_train_data['week t-%d'%(n_lag-i)] = center_c_meal_m['num_orders'].shift(n_lag-i)
    new_train_data = new_train_data.dropna()
    train_labels = new_train_data['num_orders']
    train_data = new_train_data.drop(columns=['center_id','meal_id','id','num_orders'])
    #print(train_data.head(10))
    #print(type(new_train_data))
    length = len(train_labels)
    xtest = train_data[length-n_split:]
    ytest = train_labels[length-n_split:]
    xtrain = train_data[:length-n_split]
    ytrain = train_labels[:length-n_split]
    return xtrain,ytrain,xtest,ytest

In [4]:
xtrain,ytrain,xtest,ytest = new_dataset(center_c_meal_m,6,10)

/home/sreerag/anaconda3/envs/licensePlate/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
#XGBR

eval_set =  [(xtrain, ytrain)]
xgbr = xgb.XGBRegressor(verbosity=1,booster='gbtree',eta=0.43,n_estimators=500) 
xgbr.fit(xtrain, ytrain,eval_metric="error",eval_set=eval_set)

[0]	validation_0-error:-238.29457
[1]	validation_0-error:-238.29457
[2]	validation_0-error:-238.29457
[3]	validation_0-error:-238.29457
[4]	validation_0-error:-238.29457
[5]	validation_0-error:-238.29457
[6]	validation_0-error:-238.29457
[7]	validation_0-error:-238.29457
[8]	validation_0-error:-238.29457
[9]	validation_0-error:-238.29457
[10]	validation_0-error:-238.29457
[11]	validation_0-error:-238.29457
[12]	validation_0-error:-238.29457
[13]	validation_0-error:-238.29457
[14]	validation_0-error:-238.29457
[15]	validation_0-error:-238.29457
[16]	validation_0-error:-238.29457
[17]	validation_0-error:-238.29457
[18]	validation_0-error:-238.29457
[19]	validation_0-error:-238.29457
[20]	validation_0-error:-238.29457
[21]	validation_0-error:-238.29457
[22]	validation_0-error:-238.29457
[23]	validation_0-error:-238.29457
[24]	validation_0-error:-238.29457
[25]	validation_0-error:-238.29457
[26]	validation_0-error:-238.29457
[27]	validation_0-error:-238.29457
[28]	validation_0-error:-238.2

[231]	validation_0-error:-238.29457
[232]	validation_0-error:-238.29457
[233]	validation_0-error:-238.29457
[234]	validation_0-error:-238.29457
[235]	validation_0-error:-238.29457
[236]	validation_0-error:-238.29457
[237]	validation_0-error:-238.29457
[238]	validation_0-error:-238.29457
[239]	validation_0-error:-238.29457
[240]	validation_0-error:-238.29457
[241]	validation_0-error:-238.29457
[242]	validation_0-error:-238.29457
[243]	validation_0-error:-238.29457
[244]	validation_0-error:-238.29457
[245]	validation_0-error:-238.29457
[246]	validation_0-error:-238.29457
[247]	validation_0-error:-238.29457
[248]	validation_0-error:-238.29457
[249]	validation_0-error:-238.29457
[250]	validation_0-error:-238.29457
[251]	validation_0-error:-238.29457
[252]	validation_0-error:-238.29457
[253]	validation_0-error:-238.29457
[254]	validation_0-error:-238.29457
[255]	validation_0-error:-238.29457
[256]	validation_0-error:-238.29457
[257]	validation_0-error:-238.29457
[258]	validation_0-error:-23

[459]	validation_0-error:-238.29457
[460]	validation_0-error:-238.29457
[461]	validation_0-error:-238.29457
[462]	validation_0-error:-238.29457
[463]	validation_0-error:-238.29457
[464]	validation_0-error:-238.29457
[465]	validation_0-error:-238.29457
[466]	validation_0-error:-238.29457
[467]	validation_0-error:-238.29457
[468]	validation_0-error:-238.29457
[469]	validation_0-error:-238.29457
[470]	validation_0-error:-238.29457
[471]	validation_0-error:-238.29457
[472]	validation_0-error:-238.29457
[473]	validation_0-error:-238.29457
[474]	validation_0-error:-238.29457
[475]	validation_0-error:-238.29457
[476]	validation_0-error:-238.29457
[477]	validation_0-error:-238.29457
[478]	validation_0-error:-238.29457
[479]	validation_0-error:-238.29457
[480]	validation_0-error:-238.29457
[481]	validation_0-error:-238.29457
[482]	validation_0-error:-238.29457
[483]	validation_0-error:-238.29457
[484]	validation_0-error:-238.29457
[485]	validation_0-error:-238.29457
[486]	validation_0-error:-23

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.43, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints=None,
             learning_rate=0.430000007, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=1)

In [6]:
score = xgbr.score(xtrain, ytrain)  
print("Training score: ", score)



Training score:  0.999999999950306


In [7]:
scores = cross_val_score(xgbr, xtrain, ytrain,cv=10)
print("Mean cross-validation score: %.2f" % scores.mean())

Mean cross-validation score: -0.76


In [8]:
ypred = xgbr.predict(xtest)
mse = mean_squared_error(ytest, ypred)
print("MSE: %.2f" % mse)

MSE: 2593.49


In [9]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy,predictions,test_labels


accuracy,predictions,test_labels = evaluate(xgbr,xtest,ytest)

Model Performance
Average Error: 40.4095 degrees.
Accuracy = 59.93%.


In [21]:
def create_dataset(center_c_meal_m,n_lag):
    new_train_data = center_c_meal_m
    for i in range(n_lag):
        new_train_data['week t-%d'%(n_lag-i)] = center_c_meal_m['num_orders'].shift(n_lag-i)
    new_train_data = new_train_data.dropna()
    return new_train_data



improved = pd.DataFrame()
centers = train_info['center_id'].unique()
meals =  train_info['meal_id'].unique()
#rint(centers,meals)

def improved_data(food_data,improved, n_lag, n_split):
    
    #create empty dataframe with more features
    columns = food_data.columns
    index = range(len(food_data))
    

                           
    i=0                       
    for c in centers:
        temp_center = train_info.loc[train_info['center_id']==c]
        temp_meals = temp_center['meal_id'].unique()
        for m in temp_meals:
            center_c_meal_m = temp_center.loc[temp_center['meal_id']==m]
            center_c_meal_m = create_dataset(center_c_meal_m,n_lag)          
            improved=improved.append(center_c_meal_m, ignore_index=True)
            i=i+1               
    print(i)                       
    return improved
                           
                           
new = improved_data(train_info,improved,10,10)  
print(new.head())

/home/sreerag/anaconda3/envs/licensePlate/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


3597
        id  week  center_id  meal_id  checkout_price  base_price  \
0  1047565    11         55     1885          146.53      144.53   
1  1282497    12         55     1885          145.53      146.53   
2  1136216    13         55     1885          145.50      146.50   
3  1196793    14         55     1885          142.62      141.62   
4  1324644    15         55     1885          117.37      143.62   

   emailer_for_promotion  homepage_featured  num_orders  week t-10  week t-9  \
0                      0                  0         244      177.0     323.0   
1                      0                  0         190      323.0      96.0   
2                      0                  0          68       96.0     163.0   
3                      0                  0         135      163.0     215.0   
4                      0                  0         366      215.0     285.0   

   week t-8  week t-7  week t-6  week t-5  week t-4  week t-3  week t-2  \
0      96.0     163.0     215.

In [12]:
#train test split

def new_big_dataset(improved,n_split):
    
    train_labels = improved['num_orders']
    train_data = improved.drop(columns=[ 'num_orders'])
    length = len(improved)
    for c in centers:
        temp_center = improved.loc[improved['center_id']==c]
        temp_meals = temp_center['meal_id'].unique()
        for m in temp_meals:
            d = temp_center.loc[temp_center['meal_id']==m]
            xtest = train_data[length-n_split:]
            ytest = train_labels[length-n_split:]
            xtrain = train_data[:length-n_split]
            ytrain = train_labels[:length-n_split]
    return xtrain,ytrain,xtest,ytest

xtrain,ytrain,xtest,ytest = new_big_dataset(new,10)

Empty DataFrame
Columns: []
Index: []


In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objects as go


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Test": test_labels,
    "Prediction": predictions
    
})

weeks = [*range(len(predictions))]
fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x= weeks, y=predictions, name='Predicted data',
                         line=dict(color='firebrick', width=2)))
fig.add_trace(go.Scatter(x= weeks, y=test_labels, name = 'Test data',
                         line=dict(color='royalblue', width=2)))
# Edit the layout
fig.update_layout(title='Real data VS Predicted data',
                   xaxis_title='Week',
                   yaxis_title='Number of orders')

app.layout = html.Div(children=[
    html.H1(children='Hello Sreerag'),
    
#     html.Div(["Input: ",
#               dcc.Input(id='my-input', value='initial value', type='text')]),

    html.Div(children='''
        Accuracy  = %d 
    '''%(accuracy)),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(
        port=8050,
        host='0.0.0.0')
    

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/



INFO:__main__:Dash is running on http://0.0.0.0:8050/



INFO:__main__: Warning: This is a development server. Do not use app.run_server


 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.



INFO:__main__: in production, use a production WSGI server like gunicorn instead.



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://0.0.0.0:8050/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [06/Aug/2020 17:45:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Aug/2020 17:45:03] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Aug/2020 17:45:03] "GET /_dash-layout HTTP/1.1" 200 -


In [12]:
print(type(range(3)))

<class 'range'>
